### ECG Patient Medications ###
Goals:
    - pull PatientIDs and medications from EDW
    - compare with medications from internal database
    - pull medications for all ECG patients from internal database

In [ ]:
import os
import glob
import numpy as np

import pandas as pd
from ehr.edw import Epic
from ehr.edw import ExternalIdentity
from dotenv import load_dotenv

from ehr.ehr_db import EhrDb

%load_ext autoreload
%autoreload 2
seed = 123

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# SQL Alchemy to access local database
from sqlalchemy import create_engine

In [ ]:
# Files and directories
data_root = os.path.normpath('/mnt/obi0/andreas/data/ecg')
parquet_dir = os.path.join(data_root, 'parquet')

# Database access credentials
dotenv_file = os.path.normpath('/mnt/obi0/andreas/config/credentials.env')
load_dotenv(dotenv_file)

mgh_idtype = ExternalIdentity.MGHMRN

# Epic instance
def create_epic():
    epic = Epic(edw_user=os.environ['EDW_USER'],
                edw_password=os.environ['EDW_PASSWORD'],
                db='PHS',
                out_dir=data_root)
    return epic

In [ ]:
# Concatenate all MGH ECG files
concatenated_parquet_file_name = 'MGH_RAW_FROM2001.parquet'
file_list = sorted(glob.glob(os.path.join(parquet_dir, 'MGH_RAW_meta_2*.parquet')))
def concatenate_parquet(file_list):
    df_list = []
    for f, file in enumerate(file_list):
        print(f'Loading file {f+1} of {len(file_list)}: {os.path.basename(file)}')
        df_list.append(pd.read_parquet(file))
    df = pd.concat(df_list, ignore_index=True).reset_index(drop=True)
    return df

# Load the data
#df = concatenate_parquet(file_list)
#df.to_parquet(os.path.join(parquet_dir, concatenated_parquet_file_name))
df = pd.read_parquet(os.path.join(parquet_dir, concatenated_parquet_file_name))
print(f'Loaded {len(df.file.unique())} ECGs from {len(df.PatientID.unique())} mrns.')
# Rename PatientID column: it is really an mrn
df = df.rename(columns={'PatientID': 'mrn'})
df.head()

In [ ]:
# Get a sample
np.random.seed(seed)
df_sample = df.sample(n=50).reset_index(drop=True)
sample_mrn_list = list(df_sample.mrn.unique())
print(*sample_mrn_list)

In [ ]:
epic = create_epic()
df_id = epic.patientids_from_external(external_patient_list=sample_mrn_list, 
                                      external_identity=mgh_idtype)
epic.close()
patient_id_list = list(df_id.PatientID.unique())
df_id.head(2)

In [ ]:
chunk_sizes = {'patientids': 1000,
               'medications': 1000}

epic = create_epic()
fetch_dict = epic.fetch(query_ids=patient_id_list,
                        chunk_sizes=chunk_sizes)
epic.close()

In [ ]:
# Development database
db = EhrDb(user = os.environ['EHR_DEV_USER'],
           password = os.environ['EHR_DEV_PASSWORD'],
           host='obi-cpu8',
           port='5432',
           db='ehr_dev_andreas')

In [ ]:
# Load some data into the test database
chunk_sizes = {'patientids': 1000,
               'demographics': 1000,
               'medications': 1000}
               
epic = create_epic()
db.import_epic(name='test',
               description='import test',
               protocol='2019P002257',
               query_ids=patient_id_list,
               chunk_sizes=chunk_sizes,
               epic=epic)
epic.close()

In [ ]:
chunk_sizes = {'reference_medication': 1000}
epic = Epic(edw_user=os.environ['EDW_USER'],
            edw_password=os.environ['EDW_PASSWORD'],
            db='PHS',
            out_dir=data_root)

In [ ]:
df = epic.reference_medication()